<a href="https://colab.research.google.com/github/sarkozizsofia/email_colab/blob/main/email_kantor_ilia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#                              **Kántor-Ilia emailhagyaték lektorálásra**


---



# **Általános technikai információ a colab munkafüzetről**
A colab szöveget és programkódokat tartalmazó cellákból áll. Használata "mukamenetekre oszlik", azaz a megnyitástól az ablak bezárásáig (/inaktivitás esetén időtúllépésig) "él" a virtuális gép, amelyben a különböz cellák és fájlok egymással és a felhasználóval aktív interakcióban vannak. Ha megszakad/bezáródik a munkamenet, akkor a felhasznált(feltöltött/csatlakoztatott), vagy a program által generált fájlok (esetünkben ilyen nincs) viszont eltűnnek.

**Jelen colab külső repozitóriumból** (https://github.com/digiphil-hu/kantor-ilia-email/tree/main) hívja be a szükséges **fájlokat**, melyek csak munkameneten belül elérhetőek, ezért **minden munkamenetet** a lenti cellák sorrendben való **lefuttatásával** kell kezdeni. (* futtatás = a cella bal felő sarkában található "play", magától leáll, amint végzett)

**A PROJEKT(colab) felépítése a következő:**


1) **INSTALLÁLÁS, FÁJL REPO CSATLAKOZTATÁSA**: az installálandó könyvtárakat és az emaileket tartalmazó repozitóriumot "aktiválja". Mivel az email repozitórium nem nyilvános, a cella futtatásának elindítása után a program azonnal bekéri  felhasználó github tokenjét, amelyet a megjelenő kis ablakba lehet beilleszteni, majd entert nyomva fut tovább a cella.  (ez olyan 20 mp-et vesz igénybe, ki kell várni, mielőtt a következőt futtatjuk.) **Sikeres futtatás esetén a fájlokat tartalmazó mappa a bal sávban található mappa ikonra kattintva megjelenik.**

2) **TÖRZS PROGRAMKÓD:** a program törzsét, függvényeit tartalmazza. Ezt is szükséges lefuttatni az aktuális munka előtt. (1 pillanat)

3) -tól **MUNKAÁLLOMÁSOK**: egy ilyen cella futtatásakor a program rögtön bekéri az aktuálisan vizsgálni kívánt fájl nevét a megjelenő kis mezőbe, enter megnyomása után tovább fut a program és sorban kiírja a lektoráláshoz szükséges adatokat. Ez olykor több időt (1 p) vesz igénybe, mivel egyszerre dolgozik a fájlokban, majd az ITIdata adatbázisokból is összegyűjti a szükséges adatokat.

  Ilyen "mukaállomásból plusz 2 is rendelkezésre áll, ezek használata opcionális. Így lehet 3-asával haladni, illetve lehetőséget ad arra, hogy több levelet párhuzamosan vizsgáljunk.



**1.) INSTALLÁLÁS, FÁJL REPO CSATLAKOZTATÁSA (1) **

In [ ]:
# @title
git_tok = input('Kérem a tokent: (BEILLESZT+ENTER)')
!git clone https://{git_tok}@github.com/digiphil-hu/kantor-ilia-email.git
!pip install wikidata
!pip install bs4
!pip install IPython
from google.colab import userdata
import re
import os
from pathlib import Path
from wikidata.client import Client
#----------------------------
from bs4 import BeautifulSoup
from bs4.element import NavigableString
from IPython.display import display, HTML, IFrame
# -----------------
import ipywidgets as widgets
GITREPONAME = '/content/kantor-ilia-email/'

# **2.) TÖRZS PROGRAMKÓD**


In [5]:
# @title
#----- XML --------------------------------
def from_header(bs_x):
  header = bs_x.find('teiHeader')
  head_title = header.title.text
  letter_n = header.find('publicationStmt').find('idno', {'type': 'ITIdata'}, recursive=False).text
  relations = []
  for related_i in header.find_all('relatedItem'):
    relation_type = related_i.attrs.get('type')
    if relation_type not in {'hasResponse', 'responseTo'}:
      continue
    relation_file = related_i.find('ref').attrs.get('target')
    if relation_file is not None:
      relation_file = relation_file.strip()
      relation_file = f'{GITREPONAME}{relation_file}'.strip()
    related_entity = related_i.find('idno', {'type': 'ITIdata'}).getText('NINCS KITÖLTVE').strip()
    if related_entity is None:
      related_entity = 'None'
    relations.append((relation_type, relation_file, related_entity))
  return (head_title, letter_n, relations)

def notes(bs):
  critnum = 1
  notedict = dict()
  for critic in bs.find_all('note', {'type':'critic'}):
      for t in critic.find_all():
        if t.name == 'idno':
          t.decompose()
      for t in critic.find_all():
        t.unwrap()
      critic_text = critic.text.replace('\n', ' ').replace('\t', '')
      critic_text = re.sub(r'\s+', ' ', critic_text)
      critnumstr = f'[#{str(critnum)}]'
      notedict[critnumstr] = critic_text
      critnum+=1
      critic.string = critnumstr
      critic.name = 'b'
      critic.attrs = {}
  return notedict

def get_entities_and_clear_xml(bsxml):
  for t in bsxml.find_all('supplied'):
          t.decompose()
  for gaptag in bsxml.find_all('gap'):
    gap_extent_= int(gaptag.attrs.get('extent'))
    gapunit = gaptag.attrs.get('unit')
    if gapunit == 'persName' and gaptag.parent.name == 'persName':
      gap_extent_ = 2
    gap_extent = gap_extent_*(-1)
    anonim = ' '.join(gaptag.previous_element.split(' ')[gap_extent:])

    # Store the parent reference before its contents are cleared
    parent_of_gaptag = gaptag.parent

    gaptag.string = anonim
    gaptag.name = 's'
    copycont = list(parent_of_gaptag.contents)
    parent_of_gaptag.clear() # Clear existing contents of the parent
    for c in copycont:
      if isinstance(c, str) and anonim in c:
        modified_str = c.replace(anonim, '')
        # convert the plain Python string back to a NavigableString
        parent_of_gaptag.append(bsxml.new_string(modified_str))
      else:
        parent_of_gaptag.append(c)

  critic_dict = notes(bsxml)

  entity_dict = dict()
  refnum = 1
  for person in bsxml.find('text').find_all('persName'):
          persid_tag = person.find('idno', {'type': 'ITIdata'})
          if persid_tag is not None and len(person.text.strip()) >0:
            persid = persid_tag.text
            if len(persid)==0:
              persid = "???"
              person.append(bsxml.new_string(f'[{refnum}]'))
              entity_dict[refnum] = (persid, person.text.strip())
            else:
              person.append(bsxml.new_string(f'[{refnum}]'))
              entity_dict[refnum] = (persid, person.text.replace(persid, '').strip())
            refnum+=1
            person.wrap(bsxml.new_tag('b'))
            persid_tag.decompose()
          else:
            persid = "???"
            person.append(bsxml.new_string(f'[{refnum}]'))
            person.wrap(bsxml.new_tag('b'))
            entity_dict[refnum] = (persid, person.text.strip())
            refnum+=1
  for place in bsxml.find('text').find_all('idno', {'type': 'ITIdata'}):
    place.decompose()

  bsxml.find('head').wrap(bsxml.new_tag('b'))
  for t in bsxml.find_all():
          if t.name not in {'body', 'b', 'p', 'hi', 's'}:  # 'title',
            t.unwrap()
  xml_str = str(bsxml.find('body'))
  # head
  return entity_dict, critic_dict, xml_str

def get_letter_text(xmlf):
  # kapcsolódó levelek szövegeihez
  a_xml = BeautifulSoup(xmlf, features='xml')
  for t in a_xml.find_all('supplied'):
          t.decompose()
  for qin in a_xml.find('text').find_all('idno', {'type': 'ITIdata'}):
      qin.decompose()
  for t in a_xml.find_all():
        if t.name not in {'body', 'b', 'p', 'hi'}:  # 'title',
          t.unwrap()
  b = a_xml.find('body')
  return b

#----- ITIdata --------------------------------

def get_data_from_entity_json(q_id, proplist):
    client = Client(base_url="https://itidata.abtk.hu/")
    #print(q_id)
    entity = client.get(q_id, load=True)
    q_label = entity.data['labels']['hu']['value']
    claims = entity.data['claims']
    claims_data = []
    for claim_prop in proplist:
      if claim_prop in claims.keys():
        cleardata = list()
        theclaim = entity.data['claims'][claim_prop]#[0]
        for oneclaim in theclaim:
          datavalue = oneclaim['mainsnak']['datavalue']
          datatype = oneclaim['mainsnak']['datatype']
          if datatype == 'wikibase-item':
            id_ = datavalue['value']['id']
            claim_value = q_to_label(id_)
          elif datatype == 'time':
            claim_value = datavalue['value']['time']  # +1940-08-21T00:00:00Z
            claim_value = claim_value.replace('T00:00:00Z', '').replace('+', '')
          cleardata.append(claim_value)
        cleardata = ';'.join(cleardata)
        claims_data.append(cleardata)
    return (q_label, claims_data)

def q_to_label(q):
  client = Client(base_url="https://itidata.abtk.hu/")
  entity = client.get(q, load=True)
  label = entity.data['labels'].get('hu')
  if label:
     label = label['value']
  else:
    label = entity.data['labels']['en']['value']
    print(f'### LABEL HIÁNYOS ADATLAP: {q}')
  return label

def parallel_blocks(str1, str2):
  out1 = widgets.Output(layout={'border': '10px solid white', 'background':'black'})
  out1.append_stdout(str1)
  out2 = widgets.Output(layout={'border': '10px solid black'})
  out2.append_stdout(str2)
  a = [out1, out2]
  wid=widgets.HBox(a)
  display(wid)

def blocks(str1):
  out1 = widgets.Output(layout={'border': '5px solid white', 'background':'black'})
  out1.append_stdout(str1)
  a = [out1]
  wid=widgets.HBox(a)
  display(wid)


RELATIONDICT = {'hasResponse': 'VÁLASZLEVÉL', 'responseTo': 'LEVÉL, AMELYRE VÁLASZOLT'}

def letter_proofreading(fname):
  with open(f'{GITREPONAME}{fname}', 'r') as f:
        f = f.read().replace('<gap reason="anonymised" unit="persName" extent="1"/>','<gap reason="anonymised" unit="persName" extent="1"></gap>').replace('<gap reason="anonymised" unit="persName" extent="1"/>','<gap reason="anonymised" unit="persName" extent="2"></gap>').replace('<gap reason="anonymised" unit="persName" extent="3"/>','<gap reason="anonymised" unit="persName" extent="1"></gap>').replace('<gap reason="anonymised" unit="persName" extent="4"/>','<gap reason="anonymised" unit="persName" extent="1"></gap>')
        bs_xml = BeautifulSoup(f, features='xml')
        letter_title, letter_itiid, relations_list = from_header(bs_xml)
        entity_d, notecritic_dict, xmlstr = get_entities_and_clear_xml(bs_xml)
        #print('-----------ITIdatában-------------------------------')
        itilink = f'https://itidata.abtk.hu/wiki/Item:{letter_itiid.strip()}'
        print(f'------- \033[1m \033[92m A LEVÉL SZÖVEGE ------------------------------   ADATLAP >>> {itilink}')
        print(' \n')
        display(HTML(data = xmlstr))
        print(' \n')
        print('------- \033[1m \033[93m KRITIKAI JEGYZETEK ---------------------------------------\033[0m\n')
        for ref, cnote in notecritic_dict.items():
            print(ref, cnote)
        print(' \n')
        print('-------  \033[1m \033[92m SZEMÉLYEK ---------------------------------------')
        with open('entity_data.tsv', mode='w', encoding='utf8') as out_file:
          pers_entities_str = '\n'
          for ref_num, q_n in entity_d.items():
            q, n = q_n
            if q == "???":
              q = "? AZONOSÍTÁSRA VÁRÓ SZEMÉLY ?"
              pers_entities_str += f'{ref_num}\t{n}\t -->\t{q}\n\n'
            else:
              lab, entity_prior_claims = get_data_from_entity_json(q, [ 'P22', 'P5','P4', 'P23', 'P6']) #
              pers_entities_str += f'{ref_num}\t{n}\t --> \t{lab}\t{entity_prior_claims}\n\n'
          blocks(pers_entities_str)
          out_file.write(pers_entities_str)
        print(' \n')

        if len(relations_list) > 0:
          print('------- \033[1m \033[92m KAPCSOLOLÓDÓ LEVELEK -------------------------------\n')
        else:
          print('------- \033[1m \033[93m NINCSENEK KAPCSOLOLÓDÓ LEVELEK -------------------------------\n')
        for rel in relations_list:
          reltype, rel_fn, rel_id = rel
          if len(rel_id) > 0:
            rel_itidata = q_to_label(rel_id)
          else:
            rel_itidata = 'None'
          relitilink = f'https://itidata.abtk.hu/wiki/Item:{rel_id}'
          print(RELATIONDICT[reltype], f'------FÁJLNÉV: {rel_fn}--------------------->', f'ADATLAP >>> {relitilink}\n')
          if rel_fn is not None:
            if os.path.exists(rel_fn):
              with open(rel_fn, 'r') as relf:
                rel_xml_body = get_letter_text(relf)
                display(HTML(data = str(rel_xml_body)))
            else:
              print("### A FÁJL HIÁNYZIK A REPOZITÓRIUMBÓL ###")
          else:
            print("A levélhez nem tartozik xml fájl.")

## A levelekben szereplő személyek azonosításának lektorálására vonatkozó irányelvek:

A levél szövegében kövér szedéssel és [számmal]  kiemelt nevek a szöveg alatt táblázatszerűen megismétlődnek. Itt 3 típus fordulhat elő:


1.   a név után a személy adatai is szerepelnek <--- ebben az esetben kérjük jelezni, ha TÉVES azonosítás történt, illetve kérünk pontos javaslatot. (amennyiben helyes, nincs vele teendő)
2.   a név után, az adatok helyén az *"? AZONOSÍTÁSRA VÁRÓ SZEMÉLY ?"* szöveg áll. Ebben az esetben kérünk javaslatot a személyzonosságra vonatkozóan.
3.  az *"? AZONOSÍTÁSRA VÁRÓ SZEMÉLY ?"* ignorálható, amennyiben megegyezik egy már korábban jelöl személlyel (ezen okból maradt el a jelölés ismétlése)

+. előfordulhatnak még olyan jellegű "elütések", amikor a személy neve mellett egyéb szövegrészek is "hozzáragadtak", (pl : Finta Éva szerkesztésében 2008-ban)



---


## Jelölések a levél szövegében

*   ***[szám]*** azoosított/azonosítatlan személy entitások
*   ***[#szám]*** kritikai jegyzetek, a levél szövege után találhatóak a hivatkozott megjegyzések
*   **áthúzott szövegrészek**: a nyilvános változatban rejtve lesznek. Ezek helyességére, esetleg további szavak/szövegrészek hozzáadására kérünk visszajelzést.


---


## A levél és a kapcsolódó adatok a köv. felépítésben jelennek meg:
 1. A LEVÉL SZÖVEGE   + a levél ITIdata adatlapjára mutató link
 2. KRITIKAI JEGYZETEK
 3. SZEMÉLYEK
 4. KAPCSOLOLÓDÓ LEVELEK SZÖVEGE (+fájlnév, +LINK)

    amennyiben nincs kapcsolódó levél: *"NINCSENEK KAPCSOLOLÓDÓ LEVELEK"* üzenet   : (fontos: ezen levelek szövege nem tartalmaz kiemeléseket, hivatkozásokat, mivel nincs velük feladat, csak a nyers szöveg látható a kontextus kedvéért.)


---


## A MUNKA MENETE
1. AZ 1-2 programkódokat tartalmazó cella futtatása (sorban, kivárva az eredményt, amellyel nem kell foglalkozni)
2. a következő egy vagy több munkaállomás futtatása közben megjelenő ablakba bemásolni a feldolgozni kívánt levél fájlnevét
'Ilia_Kantor_2009_03_30_07_55.xml'

(3. jegyzet a javítandókról)








In [ ]:
# @title
# MUNKAÁLLOMÁS 1 )
inp = input("Az átnézendő emailt tartalmazó fájl neve [pl Ilia_Kantor_2008_12_31_17_42.xml] (BEILLESZT + ENTER): ")
letter_proofreading(inp.strip())


In [ ]:
# @title
# MUNKAÁLLOMÁS 2 )
inp = input("Az átnézendő emailt tartalmazó fájl neve [pl Ilia_Kantor_2009_03_30_07_55.xml] (BEILLESZT + ENTER): ")
letter_proofreading(inp.strip())

In [ ]:
# @title
# MUNKAÁLLOMÁS 3 )
inp = input("Az átnézendő emailt tartalmazó fájl neve [pl Ilia_Kantor_2007_06_15_18_27.xml.xml] (BEILLESZT + ENTER): ")
letter_proofreading(inp.strip())
